# ACREAGE

## Import

- 공공데이터 포털 : [전국도시공원정보표준데이터](https://www.data.go.kr/data/15012890/standard.do) 사용

In [1]:
import pandas as pd
import re

In [2]:
try:
    df = pd.read_csv('/Users/kyeong6/Desktop/analysis/park/urban_park_data/urban_park.csv', encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv('/Users/kyeong6/Desktop/analysis/park/urban_park_data/urban_park.csv', encoding='cp949')
    except Exception as e:
        print(f"Error: {e}")

In [3]:
df.head()

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,제공기관코드,제공기관명
0,41830-00017,광탄2호 소공원,소공원,NaN,경기도 양평군 용문면 광탄리 711,37.509628,127.628406,856.0,NaN,NaN,NaN,NaN,NaN,1994-02-25,경기도 양평군청,031-770-2358,2023-05-26,4170000,경기도 양평군
1,41830-00031,광탄3호 소공원,소공원,NaN,경기도 양평군 용문면 광탄리 762,37.508443,127.627414,847.0,NaN,NaN,NaN,NaN,NaN,1994-02-25,경기도 양평군청,031-770-2358,2023-05-26,4170000,경기도 양평군
2,41830-00018,공흥 소공원,소공원,NaN,경기도 양평군 양평읍 공흥리 885-1,37.493844,127.509326,1276.0,NaN,NaN,NaN,NaN,NaN,2012-11-22,경기도 양평군청,031-770-2358,2023-05-26,4170000,경기도 양평군
3,41830-00019,한강아트로드 공원,소공원,NaN,경기도 양평군 강하면 운심리 43-25,37.496021,127.408216,3300.0,NaN,NaN,NaN,NaN,NaN,2012-08-30,경기도 양평군청,031-770-2358,2023-05-26,4170000,경기도 양평군
4,41830-00020,강하 소공원,소공원,경기도 양평군 강하면 운심길 58,경기도 양평군 강하면 운심리 28-1,37.496164,127.412326,394.0,농구장+축구장+테니스장,NaN,NaN,NaN,NaN,2012-08-30,경기도 양평군청,031-770-2358,2023-05-26,4170000,경기도 양평군


## 필요없는 속성 제거

In [4]:
# 컬럼 확인
df.columns

Index(['관리번호', '공원명', '공원구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '공원면적',
       '공원보유시설(운동시설)', '공원보유시설(유희시설)', '공원보유시설(편익시설)', '공원보유시설(교양시설)',
       '공원보유시설(기타시설)', '지정고시일', '관리기관명', '전화번호', '데이터기준일자', '제공기관코드', '제공기관명'],
      dtype='object')

In [5]:
# 필요 속성의 수가 더 적으므로 열 추출 사용
df = df[['공원명', '소재지지번주소', '공원면적', '제공기관명']]

In [6]:
df.head()

,공원명,소재지지번주소,공원면적,제공기관명
0,광탄2호 소공원,경기도 양평군 용문면 광탄리 711,856.0,경기도 양평군
1,광탄3호 소공원,경기도 양평군 용문면 광탄리 762,847.0,경기도 양평군
2,공흥 소공원,경기도 양평군 양평읍 공흥리 885-1,1276.0,경기도 양평군
3,한강아트로드 공원,경기도 양평군 강하면 운심리 43-25,3300.0,경기도 양평군
4,강하 소공원,경기도 양평군 강하면 운심리 28-1,394.0,경기도 양평군


In [7]:
# 각 도시에 따른 도시공원 면적은 제공기관명에 따른 공원 면적 확인(소재지지번주소 삭제)
df = df.drop(['소재지지번주소'], axis=1)

In [8]:
df.head()

,공원명,공원면적,제공기관명
0,광탄2호 소공원,856.0,경기도 양평군
1,광탄3호 소공원,847.0,경기도 양평군
2,공흥 소공원,1276.0,경기도 양평군
3,한강아트로드 공원,3300.0,경기도 양평군
4,강하 소공원,394.0,경기도 양평군


## 제공기관명에 따른 공원면적 크기 합, 개수 구하기

- 초기에는 "제공기관명"을 기준으로 데이터 전처리를 진행하였으나 제공기관명이 지역을 잘 나타내주지 않아 "소재지지번주소"로 변경

In [9]:
# 공원명 삭제
df = df.drop(['공원명'], axis=1)

In [10]:
df.head()

,공원면적,제공기관명
0,856.0,경기도 양평군
1,847.0,경기도 양평군
2,1276.0,경기도 양평군
3,3300.0,경기도 양평군
4,394.0,경기도 양평군


In [11]:
# groupby를 통한 "제공기관명"에 따른 공원면적 합, 갯수 확인
grouped = df.groupby('제공기관명')
result = grouped['공원면적'].agg(['sum', 'count'])

In [12]:
result

,sum,count
제공기관명,,
강원도 강릉시,1.188815e+06,86
강원도 고성군,2.692920e+05,29
강원도 동해시,4.127500e+04,5
강원도 삼척시,4.409960e+05,23
강원도 속초시,8.208620e+05,45
...,...,...
충청북도 제천시,5.787652e+05,66
충청북도 증평군,2.560390e+05,30
충청북도 진천군,8.411336e+05,36


In [13]:
result.index

Index(['강원도 강릉시', '강원도 고성군', '강원도 동해시', '강원도 삼척시', '강원도 속초시', '강원도 양구군',
       '강원도 양양군', '강원도 영월군', '강원도 원주시', '강원도 인제군',
       ...
       '충청북도 단양군', '충청북도 보은군', '충청북도 영동군', '충청북도 옥천군', '충청북도 음성군', '충청북도 제천시',
       '충청북도 증평군', '충청북도 진천군', '충청북도 청주시', '충청북도 충주시'],
      dtype='object', name='제공기관명', length=265)

In [14]:
# 그룹의 인덱스 확인
index_list = list(grouped.groups.keys())
print(index_list)

['강원도 강릉시', '강원도 고성군', '강원도 동해시', '강원도 삼척시', '강원도 속초시', '강원도 양구군', '강원도 양양군', '강원도 영월군', '강원도 원주시', '강원도 인제군', '강원도 정선군', '강원도 철원군', '강원도 춘천시', '강원도 태백시', '강원도 평창군', '강원도 홍천군', '강원도 화천군', '강원도 횡성군', '강원특별자치도 강릉시', '강원특별자치도 고성군', '강원특별자치도 동해시', '강원특별자치도 삼척시', '강원특별자치도 속초시', '강원특별자치도 양구군', '강원특별자치도 양양군', '강원특별자치도 영월군', '강원특별자치도 원주시', '강원특별자치도 인제군', '강원특별자치도 정선군', '강원특별자치도 철원군', '강원특별자치도 춘천시', '강원특별자치도 태백시', '강원특별자치도 평창군', '강원특별자치도 홍천군', '강원특별자치도 화천군', '강원특별자치도 횡성군', '경기도 가평군', '경기도 고양시', '경기도 과천시', '경기도 광명시', '경기도 광주시', '경기도 구리시', '경기도 군포시', '경기도 김포시', '경기도 남양주시', '경기도 동두천시', '경기도 부천시', '경기도 성남시', '경기도 수원시', '경기도 시흥시', '경기도 안산시', '경기도 안성시', '경기도 안양시', '경기도 양주시', '경기도 양평군', '경기도 여주시', '경기도 연천군', '경기도 오산시', '경기도 용인시', '경기도 의왕시', '경기도 의정부시', '경기도 이천시', '경기도 파주시', '경기도 평택시', '경기도 포천시', '경기도 하남시', '경기도 화성시', '경상남도 거제시', '경상남도 거창군', '경상남도 고성군', '경상남도 김해시', '경상남도 남해군', '경상남도 밀양시', '경상남도 사천시', '경상남도 산청군', '경상남도 양산시', '경상남도 의령군', '경상남도 진주시', '경상남도 창녕군', '경상남도 창원시', '경상남도 통영시', '경상남도 하동군', '경상남도 함

## 데이터 전처리 전략 수정 : 소재지지번주소 키워드로 전처리 진행

In [15]:
try:
    df = pd.read_csv('/Users/kyeong6/Desktop/analysis/park/urban_park_data/urban_park.csv', encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv('/Users/kyeong6/Desktop/analysis/park/urban_park_data/urban_park.csv', encoding='cp949')
    except Exception as e:
        print(f"Error: {e}")

In [16]:
df.head()

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,제공기관코드,제공기관명
0,41830-00017,광탄2호 소공원,소공원,NaN,경기도 양평군 용문면 광탄리 711,37.509628,127.628406,856.0,NaN,NaN,NaN,NaN,NaN,1994-02-25,경기도 양평군청,031-770-2358,2023-05-26,4170000,경기도 양평군
1,41830-00031,광탄3호 소공원,소공원,NaN,경기도 양평군 용문면 광탄리 762,37.508443,127.627414,847.0,NaN,NaN,NaN,NaN,NaN,1994-02-25,경기도 양평군청,031-770-2358,2023-05-26,4170000,경기도 양평군
2,41830-00018,공흥 소공원,소공원,NaN,경기도 양평군 양평읍 공흥리 885-1,37.493844,127.509326,1276.0,NaN,NaN,NaN,NaN,NaN,2012-11-22,경기도 양평군청,031-770-2358,2023-05-26,4170000,경기도 양평군
3,41830-00019,한강아트로드 공원,소공원,NaN,경기도 양평군 강하면 운심리 43-25,37.496021,127.408216,3300.0,NaN,NaN,NaN,NaN,NaN,2012-08-30,경기도 양평군청,031-770-2358,2023-05-26,4170000,경기도 양평군
4,41830-00020,강하 소공원,소공원,경기도 양평군 강하면 운심길 58,경기도 양평군 강하면 운심리 28-1,37.496164,127.412326,394.0,농구장+축구장+테니스장,NaN,NaN,NaN,NaN,2012-08-30,경기도 양평군청,031-770-2358,2023-05-26,4170000,경기도 양평군


In [17]:
# 필요 속성의 수가 더 적으므로 열 추출 사용
df = df[['소재지지번주소', '공원면적']]

In [18]:
df.head()

,소재지지번주소,공원면적
0,경기도 양평군 용문면 광탄리 711,856.0
1,경기도 양평군 용문면 광탄리 762,847.0
2,경기도 양평군 양평읍 공흥리 885-1,1276.0
3,경기도 양평군 강하면 운심리 43-25,3300.0
4,경기도 양평군 강하면 운심리 28-1,394.0


### "도"
- 광역시 및 전국 팔도

In [19]:
# 첫번째로 큰 개념인 "도" 개념으로 추출
keywords = ['대구광역시', '광주광역시', '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '강원도', '강원특별자치도', '경기도', '경상남도', '경상북도', '전라남도', '전라북도', '제주특별자치도', '충청남도', '충청북도']

In [20]:
# 숫자를 제거하고 키워드를 찾는 함수
def find_keyword(address):
    # 숫자 제거
    address_without_digits = re.sub(r'\d+', '', str(address))
    for keyword in keywords:
        if keyword in address_without_digits:
            return keyword
    # 키워드가 없을 경우    
    return '기타'  

In [21]:
# '소재지구주소' 열을 기반으로 키워드를 찾아 새로운 열 생성
df['광역시도'] = df['소재지지번주소'].apply(find_keyword)

In [22]:
# "광역시도"와 "소재지지번주소"를 통한 정렬
df_sorted = df.sort_values(by=['광역시도', '소재지지번주소'])

In [23]:
df_sorted

,소재지지번주소,공원면적,광역시도
3744,강원도 강릉시 강문동 41-18,5939.0,강원도
3752,강원도 강릉시 강문동 산1 일원,68044.0,강원도
3741,강원도 강릉시 견소동 산 1-3,1092.0,강원도
17457,강원도 강릉시 교동 1071-1 일원,316201.0,강원도
17459,강원도 강릉시 교동 156-11,2422.0,강원도
...,...,...,...
10242,충청북도 충주시 호암동 529,1832.1,충청북도
10292,충청북도 충주시 호암동 569,834819.1,충청북도
10249,충청북도 충주시 호암동 681-8,11402.1,충청북도
10250,충청북도 충주시 호암동 699-8,10645.1,충청북도


In [24]:
# 소재지번주소에 특정 키워드가 포함된 행들에 대해 해당 키워드에 맞는 도시공원 개수와 공원면적의 합 출력
def park_info_by_address_keyword(df, keyword):

    # 키워드 포함 확인
    filtered_df = df.dropna(subset=['소재지지번주소'])
    filtered_df = filtered_df[filtered_df['소재지지번주소'].str.contains(keyword)]

    # 행(도시공원의 갯수)과 면적합 출력
    print(f"소재지지번주소에 '{keyword}'를 포함하는 행의 개수: {len(filtered_df)}")
    if len(filtered_df) > 0:
        total_area = filtered_df['공원면적'].sum()
        print(f"소재지지번주소에 '{keyword}'를 포함하는 행들의 공원면적 합계: {total_area}")
    else:
        print("해당 키워드를 포함하는 소재지지번주소가 없습니다.")


In [25]:
# Google spreadsheet와 맞추기 위한 정렬 순서 정의
cities = [
    '대구광역시', '광주광역시', '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시',
    '강릉시', '동해시', '삼척시', '속초시', '원주시', '춘천시', '태백시', '고양시', '과천시', '광명시', '광주시', 
    '구리시', '군포시', '김포시', '남양주시', '동두천시', '부천시', '성남시', '수원시', '시흥시', '안산시', 
    '안성시', '안양시', '양주시', '여주시', '오산시', '용인시', '의왕시', '의정부시', '이천시', '파주시', 
    '평택시', '포천시', '하남시', '화성시', '거제시', '김해시', '밀양시', '사천시', '양산시', '진주시', 
    '창원시', '통영시', '경산시', '경주시', '구미시', '김천시', '문경시', '상주시', '안동시', '영주시', 
    '영천시', '포항시', '광양시', '나주시', '목포시', '순천시', '여수시', '군산시', '김제시', '남원시', 
    '익산시', '전주시', '정읍시', '서귀포시', '제주시', '계룡시', '공주시', '논산시', '당진시', '보령시', 
    '서산시', '아산시', '천안시', '제천시', '청주시', '충주시'
]

In [26]:
# 반복문을 통한 모든 도시 출력
for city in cities:
    park_info_by_address_keyword(df_sorted, city)

소재지지번주소에 '대구광역시'를 포함하는 행의 개수: 643
소재지지번주소에 '대구광역시'를 포함하는 행들의 공원면적 합계: 17887002.5
소재지지번주소에 '광주광역시'를 포함하는 행의 개수: 525
소재지지번주소에 '광주광역시'를 포함하는 행들의 공원면적 합계: 12837083.8
소재지지번주소에 '대전광역시'를 포함하는 행의 개수: 454
소재지지번주소에 '대전광역시'를 포함하는 행들의 공원면적 합계: 14691353.499999998
소재지지번주소에 '부산광역시'를 포함하는 행의 개수: 705
소재지지번주소에 '부산광역시'를 포함하는 행들의 공원면적 합계: 31016085.83
소재지지번주소에 '서울특별시'를 포함하는 행의 개수: 1898
소재지지번주소에 '서울특별시'를 포함하는 행들의 공원면적 합계: 98474779.53
소재지지번주소에 '세종특별자치시'를 포함하는 행의 개수: 99
소재지지번주소에 '세종특별자치시'를 포함하는 행들의 공원면적 합계: 1313251.0
소재지지번주소에 '울산광역시'를 포함하는 행의 개수: 467
소재지지번주소에 '울산광역시'를 포함하는 행들의 공원면적 합계: 21729413.6
소재지지번주소에 '인천광역시'를 포함하는 행의 개수: 744
소재지지번주소에 '인천광역시'를 포함하는 행들의 공원면적 합계: 28688652.952999998
소재지지번주소에 '강릉시'를 포함하는 행의 개수: 172
소재지지번주소에 '강릉시'를 포함하는 행들의 공원면적 합계: 2377629.5999999996
소재지지번주소에 '동해시'를 포함하는 행의 개수: 10
소재지지번주소에 '동해시'를 포함하는 행들의 공원면적 합계: 82550.0
소재지지번주소에 '삼척시'를 포함하는 행의 개수: 46
소재지지번주소에 '삼척시'를 포함하는 행들의 공원면적 합계: 881992.0
소재지지번주소에 '속초시'를 포함하는 행의 개수: 90
소재지지번주소에 '속초시'를 포함하는 행들의 공원면적 합계: 1641724.0
소재지지번주소에 '원주시'를 포함하는 행의 